# Functions & Data

In [1]:
from pathlib import Path
from collections import defaultdict
from operator import itemgetter
import json

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import minimum_spanning_tree
from networkx.algorithms.approximation import traveling_salesman_problem


TDATA = Path.home() / 'source' / 'repos' / 'OpenFusion' / 'tdata'
IZDATA = Path('..') / 'IZScoreResearch' / 'izscore' / 'data'
MINIMAP = plt.imread('../FFDump/tutorialassets/all_minimap.png')
TILE_SIZE = 51200
TILE_COUNT = 16
MINIMAP_SIZE = MINIMAP.shape[0]
XY_DIST_THRES = TILE_SIZE / 4
Z_DIST_THRES = TILE_SIZE / 10
CRATE_TYPE_ID = 9
TIME_TO_LV36 = 6 * 24 * 60 * 60.0
MISSION_TYPES = [
    'Mission',
    'Guide Mission',
    'Nano Mission',
    'World Mission',
]
ITEM_TABLES = [
    'm_pWeaponItemTable',
    'm_pShirtsItemTable',
    'm_pPantsItemTable',
    'm_pShoesItemTable',
    'm_pHatItemTable',
    'm_pGlassItemTable',
    'm_pBackItemTable',
    'm_pGeneralItemTable',
    '',
    'm_pChestItemTable',
    'm_pVehicleItemTable',
]
TRADABILITY = ['Not Tradable', 'Tradable']
GENDER = ['Any', 'Boy', 'Girl']
RARITY = ['Any', 'Common', 'Uncommon', 'Rare', 'Ultra Rare']
EVENTS = {
    0: 'No Event',
    1: 'Knishmas',
    2: 'Halloween',
    3: 'Easter',
    # The rest here are unofficial
    4: 'Birthday Bash',
    5: "Valentine's Day",
    6: "St. Patrick's Day",
    7: 'April Fools',
    8: "Mother's Day",
    9: 'Independence Day',
    10: 'Thanksgiving',
    -1: 'Custom Event',
}

In [2]:
class MergeDict(defaultdict):
    def __init__(self, *args, fitness_key = 'approx_taros', **kwargs):
        super().__init__(self.default_obj, *args, **kwargs)
        self.fitness_key = fitness_key

    def default_obj(self):
        return {self.fitness_key: np.inf}

    def merge(self, key, obj, fitness_key=None):
        self[key] = min([self[key], obj], key=itemgetter(fitness_key or self.fitness_key))
    
    def merge_from(self, other, fitness_key=None):
        if not isinstance(other, MergeDict):
            raise ValueError('Can only merge (add) MergeDicts with other MergeDicts.')

        for key, obj in other.items():
            self.merge(key, obj, fitness_key=fitness_key)

        self.fitness_key = fitness_key or self.fitness_key


def patch(base_obj, patch_obj):
    for key, value in patch_obj.items():
        if key[0] == '!':
            base_obj[key[1:]] = value
            continue
        
        if key in base_obj:
            if value is None:
                del base_obj[key]
            elif not isinstance(value, (dict, list)):
                base_obj[key] = value
            elif isinstance(value, list):
                base_obj[key].extend(value)
            else:
                patch(base_obj[key], value)
        else:
            base_obj[key] = value


def get_patched(name, patch_name='1013'):
    with open(TDATA / f'{name}.json') as r:
        base_obj = json.load(r)
    
    if patch_name:
        patch_path = TDATA / 'patch' / patch_name / f'{name}.json'

        if patch_path.is_file():
            with open(patch_path) as r:
                patch_obj = json.load(r)
            patch(base_obj, patch_obj)
    
    return base_obj


def mapify_drops(drops):
    keymap = {
        'Racing': 'EPID',
        'NanoCapsules': 'Nano',
        'CodeItems': 'Code',
    }

    drop_maps = {}
    for key in drops:
        real_key = keymap.get(key, key[:-1] + 'ID')
        drop_maps[key] = {obj[real_key]: obj for obj in drops[key].values()}

    return drop_maps


def pixel(x):
    return x / TILE_COUNT / TILE_SIZE * MINIMAP_SIZE


def rpixel(y):
    return MINIMAP_SIZE - 1 - y / TILE_COUNT / TILE_SIZE * MINIMAP_SIZE


def extract_coordinates(focmob):
    x = np.array([m['iX'] for m in focmob]).astype(float)
    y = np.array([m['iY'] for m in focmob]).astype(float)
    z = np.array([m['iZ'] for m in focmob]).astype(float)
    ins = np.array([m.get('iMapNum', 0) for m in focmob])
    return x, y, z, ins


def get_masked_dists(x, y, z, ins, xy_dist_thres=XY_DIST_THRES, z_dist_thres=Z_DIST_THRES):
    dist = np.sqrt(np.abs((x.reshape(-1, 1) - x.reshape(1, -1)) ** 2 + (y.reshape(-1, 1) - y.reshape(1, -1)) ** 2))
    zdist = np.abs(z.reshape(-1, 1) - z.reshape(1, -1))
    diff_ins = ins.reshape(-1, 1) != ins.reshape(1, -1)

    zero_mask = (dist >= xy_dist_thres) | (zdist >= z_dist_thres) | diff_ins
    dist[zero_mask] = 0.0
    zdist[zero_mask] = 0.0

    return dist, zdist


def get_tsp_ordering(dist, zdist):
    g = nx.Graph(np.sqrt((dist ** 2) + (zdist ** 2)))
    return [(traveling_salesman_problem(g, nodes=cc) if len(cc) > 1 else list(cc)) for cc in nx.connected_components(g)]


def get_mst(dist, zdist):
    mst = minimum_spanning_tree(csr_matrix(np.sqrt((dist ** 2) + (zdist ** 2)))).toarray()

    id_tups = list(zip(*np.nonzero(mst)))
    g = nx.Graph(id_tups)
    return [(list(conns), [(i, j) for i, j in id_tups if i in conns]) for conns in nx.connected_components(g)]


def kill_time(mob_order, mob_hp, mob_level, attack_power, defense, delay, boosted):
    damage = (attack_power ** 2) / (attack_power + defense)
    damage = max(10 + attack_power / 10, damage - (defense - attack_power / 6) * mob_level / 100)
    damage *= 5 / 4
    damage += damage * float(boosted) / 4
    mob_hp /= (1 + float(mob_order % 3 == 0))
    return (np.ceil(mob_hp / damage) - 1) * delay

 
def loop_time(tsp, dist, mobs, boosted=False):
    attack_power = 761 if mobs[0]['bGroup'] else 772
    delay = 1.0 if mobs[0]['bGroup'] else 0.8
    player_avg_speed = 750
    
    prev = tsp[0]
    kill_total = kill_time(0, mobs[prev]['iHP'], mobs[prev]['iLevel'], attack_power, mobs[prev]['iDefense'], delay, boosted)
    delay_total = 0.0
    resp_time = mobs[prev]['iRespawnTime'] / 10
    threshold = kill_total + resp_time if resp_time < 1e7 else kill_total + 3 * 60.0

    for k in range(1, len(tsp)):
        cur = tsp[k]
        delay_total += dist[cur, prev] / player_avg_speed 
        if k < len(tsp) - 1:
            kill_total += kill_time(k, mobs[cur]['iHP'], mobs[cur]['iLevel'], attack_power, mobs[cur]['iDefense'], delay, boosted)
        prev = cur

    if mobs[0]['bGroup']:
        kill_total /= 2.5

    return max(kill_total + delay_total, threshold)
        

def best_loop(tsps, dist, mobs, boosted=False):
    return max([(tsp, loop_time(tsp, dist, mobs, boosted)) for tsp in tsps], key=lambda t: max(1, len(t[0]) - 1) / t[1])


In [3]:
mobs = get_patched('mobs')
drops = get_patched('drops')
eggs = get_patched('eggs')
NPCs = get_patched('NPCs')
paths = get_patched('paths')

with open(TDATA / 'xdt1013.json') as r:
    xdt = json.load(r)

with open(IZDATA / 'merged_scores.json') as r:
    score_dict = json.load(r)

with open(IZDATA / 'ogracingpatch' / 'drops.json') as r:
    og_racing_dict = json.load(r)
    
patch(drops, og_racing_dict)
drop_maps = mapify_drops(drops)

# NPC maps
npc_dict = {d['m_iNpcNumber']: {**d, 'm_strName': xdt['m_pNpcTable']['m_pNpcStringData'][d['m_iNpcName']]['m_strName']} for d in xdt['m_pNpcTable']['m_pNpcData']}
loaded_npcs = {n['iNPCType'] for n in NPCs['NPCs'].values()}

# Egg maps
egg_type_dict = {et['Id']: {**et, 'm_strName': '{m_strName} {m_strComment1}'.format(**xdt['m_pShinyTable']['m_pShinyStringData'][et['Id']])} for et in eggs['EggTypes'].values()}
egg_groups = defaultdict(list)

for egg in eggs['Eggs'].values():
    egg_id = egg['iType']

    if egg_id not in egg_type_dict:
        continue

    egg_name = egg_type_dict[egg_id]['m_strName']
    egg_groups[egg_id].append({
        **egg,
        'iMapNum': 0,
        'iHP': 1,
        'iLevel': 1,
        'iDefense': 1,
        'iRespawnTime': egg_type_dict[egg_id]['Regen'] * 10.0,
        'bGroup': False,
        'bBoss': False,
    })

# Instance maps
instance_list = [
    {
        **ins,
        'm_strName': xdt['m_pInstanceTable']['m_pWarpNameData'][ins['m_iInstanceNameID']]['m_pstrNameString'],
    } for ins in xdt['m_pInstanceTable']['m_pInstanceData']
]
ep_score_thres = {
    racing['EPID']: list(zip([racing['ScoreCap']] + racing['RankScores'], racing['RankScores']))
    for racing in drop_maps['Racing'].values()
}
ep_best_times = {
    score_info['epid']: [sorted([obj for obj in score_info['scores'] if obj['score'] < up_limit and obj['score'] >= down_limit], key=itemgetter('time'))
                         for up_limit, down_limit in ep_score_thres[score_info['epid']]]
    for ep_name, score_info in score_dict.items()
    if not ep_name.endswith('(Old)')
}

# Mission maps
mission_task_dict = defaultdict(list)
for m in xdt['m_pMissionTable']['m_pMissionData'][1:]:
    mission_task_dict[m['m_iHMissionID']].append({
        **m,
        'm_strName': xdt['m_pMissionTable']['m_pMissionStringData'][m['m_iHMissionName']]['m_pstrNameString'].replace('\n', ' '),
        'rewards': [(iid, itype) 
                    for iid, itype in zip(xdt['m_pMissionTable']['m_pRewardData'][m['m_iSUReward']]['m_iMissionRewardItemID'], xdt['m_pMissionTable']['m_pRewardData'][m['m_iSUReward']]['m_iMissionRewarItemType'])
                    if iid > 0],
    })
mission_dict = {mid: max(tasks, key=lambda o: len(o['rewards'])) for mid, tasks in mission_task_dict.items()}
reward_to_task = defaultdict(list)
for mid, task in mission_dict.items():
    for reward_key in task['rewards']:
        reward_to_task[reward_key].append(task)
growth_arr = np.array([g['m_iReqBlob_NanoCreate'] for g in xdt['m_pAvatarTable']['m_pAvatarGrowData']])
growth_frac = growth_arr.cumsum() / growth_arr.sum()

# Item maps
item_dict = {(d['m_iItemNumber'], i): {**d, **xdt[table]['m_pItemStringData'][d['m_iItemName']]}
             for i, table in enumerate(ITEM_TABLES)
             if table != ''
             for d in xdt[table]['m_pItemData']}

# ItemReference maps
itemref_to_id_type = {irid: (ir['ItemID'], ir['Type']) for irid, ir in drop_maps['ItemReferences'].items()}
real_gender_map = {irid: item_dict[itemref_to_id_type[irid]].get('m_iReqSex', 0) for irid in drop_maps['ItemReferences']}
real_rarity_map = {irid: item_dict[itemref_to_id_type[irid]].get('m_iRarity', 0) for irid in drop_maps['ItemReferences']}

itemset_view_maps = {
    isid: np.array([
        [
            [
                (
                    itemset['AlterItemWeightMap'].get(str(irid), itemset['DefaultItemWeight'])
                    if ((itemset['IgnoreGender'] or itemset['AlterGenderMap'].get(str(irid), real_gender_map[irid]) in [0, gender_id])
                        and (itemset['IgnoreRarity'] or itemset['AlterRarityMap'].get(str(irid), real_rarity_map[irid]) in [0, rarity_id]))
                    else 
                    0
                ) for irid in itemset['ItemReferenceIDs']
            ] for rarity_id in range(1, 5)
        ] for gender_id in range(1, 3)
    ], dtype=np.float64) for isid, itemset in drop_maps['ItemSets'].items()
}

# Mob maps
bossmap = {
    443: 55,
    447: 253,
    449: 61,
    450: 256,
    451: 257,
    452: 64,
    453: 65,
    456: 68,
    457: 263,
    458: 264,
    70: 264,  # A stray hydra is registered as a separate mob, let it slide
    459: 71,  # Sewer Creepers drop level 9 crates in the future?!
    462: 74,
    463: 75,
    464: 270,
    465: 77,
    469: 275,
    475: 281,  # Shocktangler level 4 crate in Sector V Future ?!
    477: 283,
    478: 90,
    479: 285,
    480: 286,
    481: 287,
    93: 287,  # two separate Mad Mowers in the same area, let it slide
    482: 94,
    483: 289,
    484: 96,
    487: 293,
    491: 297,
    492: 104,
    493: 299,
    494: 300,
    495: 107,
    496: 302,
    500: 306,
    503: 309,
    504: 310,
    509: 121,
    510: 316,
    511: 317,
    555: 167,
    589: 201,
    1268: 1267,  # Lair-specific boss-normal pairing Maelstrom Creeper
    # 2111: 2082,  # we probably don't want to match Hyper Fusionfly with Fusionfly
    # 3038: 2464,  # same here, Stalactitan and the boss have different drop tables
    3374: 3373,
    3541: 3540,
}
for mob in mobs['groups'].values():
    mobid = mob['iNPCType']
    followids = [m['iNPCType'] for m in mob['aFollowers'] if m['iNPCType'] != mobid]
    if followids and mobid not in bossmap:
        key = followids[0]
        bossmap[mobid] = key
# we don't want to match Father Freakosaurus
del bossmap[2108]
for mobid, key in sorted(bossmap.items()):
    print(xdt['m_pNpcTable']['m_pNpcStringData'][key]['m_strName'], key, 'mobs have a boss', xdt['m_pNpcTable']['m_pNpcStringData'][mobid]['m_strName'], mobid)

mobmap = {}
for typed_map in mobs.values():
    for mob in typed_map.values():
        mobid = mob['iNPCType']
        key = bossmap.get(mobid, mobid)

        if key not in mobmap:
            mobmap[key] = {
                'color': (np.random.random(), np.random.random(), np.random.random()),
                'mobs': [],
            }
        mobmap[key]['mobs'].append({
            **mob,
            'iHP': npc_dict[mobid]['m_iHP'],
            'iLevel': npc_dict[mobid]['m_iNpcLevel'],
            'iDefense': npc_dict[mobid]['m_iProtection'],
            'iRespawnTime': npc_dict[mobid]['m_iRegenTime'],
            'bGroup': 'aFollowers' in mob,
            'bBoss': key != mobid,
        })

        if 'aFollowers' in mob:
            for mobsat in mob['aFollowers']:
                mobsatid = mobsat['iNPCType']

                if mobsatid not in mobmap:
                    mobmap[mobsatid] = {
                        'color': (np.random.random(), np.random.random(), np.random.random()),
                        'mobs': [],
                    }
                mobmap[mobsatid]['mobs'].append({
                    'iNPCType': mobsatid,
                    'iX': mob['iX'] + mobsat['iOffsetX'],
                    'iY': mob['iY'] + mobsat['iOffsetY'],
                    'iZ': mob['iZ'],
                    'iMapNum': mob.get('iMapNum', 0),
                    'iHP': npc_dict[mobsatid]['m_iHP'],
                    'iLevel': npc_dict[mobsatid]['m_iNpcLevel'],
                    'iDefense': npc_dict[mobsatid]['m_iProtection'],
                    'iRespawnTime': npc_dict[mobsatid]['m_iRegenTime'],
                    'bGroup': True,
                    'bBoss': False,
                })

# Temporary taros setting
taros_per_second = 30

Dire Hydra 264 mobs have a boss Dire Hydra 70
Mad Mower 287 mobs have a boss Mad Mower 93
Fusion Spawn 55 mobs have a boss Jumbo Fusion Spawn 443
Tech Queen 253 mobs have a boss Tech Queen Widow 447
Air Drone 61 mobs have a boss Air Drone Enforcer 449
Road Golem 256 mobs have a boss General Golem 450
Noxious Spawn 257 mobs have a boss Sickly Noxoius Spawn 451
Bad Burro 64 mobs have a boss Big Bad Burro 452
Candy Bandit 65 mobs have a boss Candy Bandit Captain 453
Painsaw 68 mobs have a boss Bossblade Painsaw 456
Newsprint Ninja 263 mobs have a boss Newsprint Ninja Boss 457
Dire Hydra 264 mobs have a boss Alpha Hydra 458
Sewer Creeper 71 mobs have a boss Sewer Stalker 459
Dread Head 74 mobs have a boss Don Dread Head 462
Motor Raptor 75 mobs have a boss Motor Master 463
Toxic Spawn 270 mobs have a boss Highly Toxic Spawn 464
Tank Terror 77 mobs have a boss Giant Tank Terror 465
Ultramagno Beetle 275 mobs have a boss Charged Magno Beetle 469
Shocktangler 281 mobs have a boss Electrotangl

In [23]:
def item_metadata(key):
    iid, itype = key
    return {
        'id': iid,
        'type': ITEM_TABLES[itype][3:-9],
        'name': (item_dict[key]['m_strName'].strip()
                 if itype != CRATE_TYPE_ID or len(item_dict[key]['m_strComment']) > 20 else
                 item_dict[key]['m_strName'].strip() + ' ' + item_dict[key]['m_strComment'].strip()),
        'level': item_dict[key].get('m_iMinReqLev', 0),
        'tradable': TRADABILITY[item_dict[key]['m_iTradeAble']],
        'rarity': RARITY[item_dict[key].get('m_iRarity', 0)],
        'gender': GENDER[item_dict[key].get('m_iReqSex', 0)],
    }


def seconds_to_taros(seconds):
    return round(seconds) * taros_per_second


def taros_to_seconds(taros):
    return taros / taros_per_second


def fastest_crates():
    perf_list = []

    for mob_id, mob_info in mobmap.items():
        moblist = mob_info['mobs']
        x, y, z, ins = extract_coordinates(moblist)
        dist, zdist = get_masked_dists(x, y, z, ins)
        tsps = get_tsp_ordering(dist, zdist)
        perf_list.append((mob_id, *best_loop(tsps, dist, moblist)))

    return max(perf_list, key=lambda t: max(1, len(t[1]) - 1) / t[2])


def fastest_miscs():
    secs_per_miscs = MergeDict(fitness_key='time_till_misc')

    for mob_id, mob_info in mobmap.items():
        moblist = mob_info['mobs']
        x, y, z, ins = extract_coordinates(moblist)
        dist, zdist = get_masked_dists(x, y, z, ins)
        tsps = get_tsp_ordering(dist, zdist)
        tsp, mob_loop_time = best_loop(tsps, dist, moblist)
        mob_count = max(1, len(tsp) - 1)

        if mob_id not in drop_maps['Mobs']:
            continue

        mob_obj = drop_maps['Mobs'][mob_id]
        mob_drop_id = mob_obj['MobDropID']

        secs_per_miscs.merge_from(seconds_per_misc(mob_count, mob_loop_time, mob_drop_id, 
                                                   src_id=mob_id, src_name=npc_dict[mob_id]['m_strName']))
        
    return secs_per_miscs


def seconds_per_misc(mob_count, mob_loop_time, mob_drop_id, src_id, src_name):
    if mob_drop_id not in drop_maps['MobDrops']:
        return MergeDict()

    mob_drop_obj = drop_maps['MobDrops'][mob_drop_id]

    # first calculate how much the crate outputs sell for (taros)
    sell_per_sec = 0.0

    crate_chance_obj = drop_maps['CrateDropChances'][mob_drop_obj['CrateDropChanceID']]
    crate_type_obj = drop_maps['CrateDropTypes'][mob_drop_obj['CrateDropTypeID']]
    
    dc = crate_chance_obj['DropChance']
    dct = crate_chance_obj['DropChanceTotal']

    # only one level deep is enough
    for irid, best_chance in drops_per_crate_group(crate_type_obj['CrateIDs'], crate_chance_obj['CrateTypeDropWeights']).items():
        kill_per_sec = (dc * mob_count) / (dct * mob_loop_time)
        item_info = item_dict[itemref_to_id_type[irid]]
        sell_per_sec += item_info['m_iSellAble'] * item_info['m_iItemSellPrice'] * kill_per_sec * best_chance

    # now fetch the actual misc chances and amounts
    misc_chance_obj = drop_maps['MiscDropChances'][mob_drop_obj['MiscDropChanceID']]
    misc_type_obj = drop_maps['MiscDropTypes'][mob_drop_obj['MiscDropTypeID']]

    secs_per_misc = MergeDict(fitness_key='time_till_misc')

    for name in ['Potion', 'Boost', 'Taro', 'FM']:
        chance = misc_chance_obj[name + 'DropChance']
        chance_total = misc_chance_obj[name + 'DropChanceTotal']
        amount = misc_type_obj[name + 'Amount']

        misc_per_sec = (amount * chance * mob_count) / (chance_total * mob_loop_time)
        if name == 'Taro':
            misc_per_sec += sell_per_sec

        if misc_per_sec > 1e-7:
            secs_per_misc.merge(name, {
                'name': name,
                'src_id': src_id,
                'src_name': src_name,
                'srcs_in_loop': mob_count,
                'src_loop_time': mob_loop_time,
                'time_till_misc': 1 / misc_per_sec,
            })

    return secs_per_misc


def seconds_per_droppable_crates(mob_count, mob_loop_time, mob_drop_id, src_id, src_name):
    if mob_drop_id not in drop_maps['MobDrops']:
        return MergeDict()

    mob_drop_obj = drop_maps['MobDrops'][mob_drop_id]
    crate_chance_obj = drop_maps['CrateDropChances'][mob_drop_obj['CrateDropChanceID']]
    crate_type_obj = drop_maps['CrateDropTypes'][mob_drop_obj['CrateDropTypeID']]

    crate_drop_weights = crate_chance_obj['CrateTypeDropWeights']
    crate_ids = crate_type_obj['CrateIDs']
    crate_objs = [drop_maps['Crates'].get(crate_id) for crate_id in crate_ids]
    
    dc = crate_chance_obj['DropChance']
    dct = crate_chance_obj['DropChanceTotal']
    cdwt = sum(crate_drop_weights)

    secs_per_crates = MergeDict(fitness_key='time_till_item')

    for crate, cdw in zip(crate_objs, crate_drop_weights):
        if crate is None:
            continue

        crate_id = crate['CrateID']
        num = dct * cdwt * mob_loop_time
        denom = dc * cdw * mob_count 

        if denom > 1e-7:
            time_till_item = num / denom
            key = (crate_id, CRATE_TYPE_ID)
            
            secs_per_crates.merge(key, {
                **item_metadata(key),
                'src_id': src_id,
                'src_name': src_name,
                'srcs_in_loop': mob_count,
                'src_loop_time': mob_loop_time,
                'time_till_item': time_till_item,
                'approx_taros': seconds_to_taros(time_till_item),
            })

    for irid, best_chance in drops_per_crate_group(crate_ids, crate_drop_weights).items():
        key = itemref_to_id_type[irid]

        num = dct * mob_loop_time
        denom = dc * mob_count * best_chance

        if denom > 1e-7:
            time_till_item = num / denom
            
            secs_per_crates.merge(key, {
                **item_metadata(key),
                'src_id': src_id,
                'src_name': src_name,
                'srcs_in_loop': mob_count,
                'src_loop_time': mob_loop_time,
                'time_till_item': time_till_item,
                'approx_taros': seconds_to_taros(time_till_item),
            })

    return secs_per_crates


def seconds_per_mob_crates(mob_count, mob_loop_time, mob_id):
    if mob_id not in drop_maps['Mobs']:
        return MergeDict()

    mob_obj = drop_maps['Mobs'][mob_id]

    return seconds_per_droppable_crates(mob_count, mob_loop_time, mob_obj['MobDropID'],
                                        src_id=mob_id, src_name=('Mob: ' + npc_dict[mob_id]['m_strName']))


def drops_per_crate(crate_id):
    if crate_id not in drop_maps['Crates']:
        return {}
    
    crate = drop_maps['Crates'][crate_id]
    rarity_weights = drop_maps['RarityWeights'][crate['RarityWeightID']]['Weights']
    itemset = drop_maps['ItemSets'][crate['ItemSetID']]

    rarity_weights_arr = np.array(rarity_weights + [0 for _ in range(4 - len(rarity_weights))], dtype=np.float64)
    rarity_weights_gendered_arr = np.vstack((rarity_weights_arr, rarity_weights_arr))

    itemset_view_arr = itemset_view_maps[itemset['ItemSetID']]
    rarity_filtered_arr = rarity_weights_gendered_arr * (itemset_view_arr.sum(axis=-1) > 0)
    rarity_odds_arr = rarity_filtered_arr / rarity_filtered_arr.sum(axis=1, keepdims=True)

    itemset_odds_arr = itemset_view_arr / np.maximum(1.0, itemset_view_arr.sum(axis=-1, keepdims=True))
    itemset_gender_arr = np.sum(itemset_odds_arr * rarity_odds_arr[:, :, np.newaxis], axis=1)
    itemset_best_arr = itemset_gender_arr.max(axis=0)
    return dict(zip(itemset['ItemReferenceIDs'], itemset_best_arr))


def drops_per_crate_group(crate_ids, crate_weights):
    cwt = sum(crate_weights)
    best_chances = defaultdict(float)

    for crate_id, crate_weight in zip(crate_ids, crate_weights):
        for irid, best_chance_crate in drops_per_crate(crate_id).items():
            best_chances[irid] += best_chance_crate * crate_weight / cwt

    return best_chances


def seconds_per_mob_drops():
    secs_per_drops = MergeDict(fitness_key='time_till_item')

    for mob_id, mob_info in mobmap.items():
        moblist = mob_info['mobs']
        x, y, z, ins = extract_coordinates(moblist)
        dist, zdist = get_masked_dists(x, y, z, ins)
        tsps = get_tsp_ordering(dist, zdist)

        # do not use best_loop, boss groups in lairs etc. may be more profitable if their drop rates are altered
        for tsp in tsps:
            mob_loop_time = loop_time(tsp, dist, moblist)
            mob_count = max(1, len(tsp) - 1)
            secs_per_drops.merge_from(seconds_per_mob_crates(mob_count, mob_loop_time, mob_id))

    return secs_per_drops


def seconds_per_event_drops():
    secs_per_event_drops = MergeDict(fitness_key='time_till_item')

    mob_id, tsp, mob_loop_time = fastest_crates()
    mob_count = max(1, len(tsp) - 1)
    print('Fastest CRATE rate:', mob_id, npc_dict[mob_id]['m_strName'], mob_loop_time / mob_count, 'seconds per CRATE')

    for event in drop_maps['Events'].values():
        event_id = event['EventID']
        secs_per_event_drops.merge_from(seconds_per_droppable_crates(mob_count, mob_loop_time, event['MobDropID'],
                                                                     src_id=event_id, src_name=('Event: ' + EVENTS.get(event_id, EVENTS[-1]))))

    return secs_per_event_drops


def taros_per_code_items():
    t_per_code_items = MergeDict(fitness_key='approx_taros')

    for code_item in drop_maps['CodeItems'].values():
        for irid in code_item['ItemReferenceIDs']:
            key = itemref_to_id_type[irid]
            src_loop_time = growth_frac[4] * TIME_TO_LV36

            t_per_code_items.merge(key, {
                **item_metadata(key),
                'src_id': 0,
                'src_name': 'Code: ' + code_item['Code'],
                'srcs_in_loop': 1,
                'src_loop_time': src_loop_time,
                'time_till_item': src_loop_time,
                'approx_taros': seconds_to_taros(src_loop_time),
            })

    return t_per_code_items


def taros_per_vendor_items():
    t_per_vendor_items = MergeDict(fitness_key='approx_taros')

    for obj in xdt['m_pVendorTable']['m_pItemData'][1:]:
        key = (obj['m_iitemID'], obj['m_iItemType'])
        approx_taros = item_dict[key]['m_iItemPrice']
        npc_id = obj['m_iNpcNumber']

        if key[0] > 0 and npc_id in loaded_npcs:
            t_per_vendor_items.merge(key, {
                **item_metadata(key),
                'src_id': npc_id,
                'src_name': 'Vendor: ' + npc_dict[npc_id]['m_strName'],
                'srcs_in_loop': 1,
                'src_loop_time': 1.0,
                'time_till_item': taros_to_seconds(approx_taros),
                'approx_taros': approx_taros,
            })

    return t_per_vendor_items


def seconds_per_egg_items():
    secs_per_egg_items = MergeDict(fitness_key='time_till_item')

    for egg_id, egglist in egg_groups.items():
        egg_type = egg_type_dict[egg_id]
        key = (egg_type['DropCrateId'], CRATE_TYPE_ID)

        if key[0] == 0:
            continue

        x, y, z, ins = extract_coordinates(egglist)
        dist, zdist = get_masked_dists(x, y, z, ins, xy_dist_thres=(TILE_SIZE * 10), z_dist_thres=TILE_SIZE)
        tsps = get_tsp_ordering(dist, zdist)
        tsp, egg_loop_time = best_loop(tsps, dist, egglist)
        egg_count = max(1, len(tsp) - 1)
        
        time_till_item = egg_loop_time / egg_count
        secs_per_egg_items.merge(key, {
            **item_metadata(key),
            'src_id': egg_id,
            'src_name': 'Egg: ' + egg_type['m_strName'],
            'srcs_in_loop': egg_count,
            'src_loop_time': egg_loop_time,
            'time_till_item': time_till_item,
            'approx_taros': seconds_to_taros(time_till_item),
        })

    return secs_per_egg_items


def seconds_per_racing_crates():
    secs_per_racing_crates = MergeDict(fitness_key='time_till_item')

    for racing in drop_maps['Racing'].values():
        epid = racing['EPID']

        time_multps = [5.0, 2.0, 1.0, 1.0, 1.0]

        for crate_id, ep_record_list, time_multp in reversed(list(zip(racing['Rewards'], ep_best_times[epid], time_multps))):
            if crate_id <= 0:
                continue

            key = (crate_id, CRATE_TYPE_ID)
            src_loop_time = ep_record_list[0]['time'] * time_multp

            # spec: 
            # gold crate items are harder, so much so that people don't think silver and bronze rewards matter
            # you don't necessarily get gold crate items faster (you might, but you can miss pods and get faster silver & bronze crates too)
            # gist is: there's high noise in the correlation between crate acquire time and taro opportunity cost
            # crate time = above-average speed race to acquire it + the time it takes to master the skill
            # time it takes to master the skill is a function of IZ hardness and crate type
            # time of the race to acquire a crate is a function of crate type, crate threshold, score function, pod hardness (or how much time each +1 pod would cost), and some player data or some assumption about player skill
            # idea: pick avg player starting point, direction and a step size, keep decaying the step size and arrive at a point -> inf. calculate wrt that point.
            # idea: pick starting point and end point, and a step size, count how many decaying steps it takes to get there, that's the hardness multiplier

            # action: lets just pick the fastest race in gold/silver/bronze categories in record. starting from bronze, do the grad. ascent technique and count steps, then add the time it would take to take all those steps (* 0.9 maybe to weight it)

            secs_per_racing_crates.merge(key, {
                **item_metadata(key),
                'src_id': epid,
                'src_name': 'Racing: ' + racing['EPName'],
                'srcs_in_loop': 1,
                'src_loop_time': src_loop_time,
                'time_till_item': src_loop_time,
                'approx_taros': seconds_to_taros(src_loop_time),
            })

    return secs_per_racing_crates


def seconds_per_mission_rewards():
    secs_per_mission_rewards = MergeDict(fitness_key='time_till_item')

    for key, tasks in reward_to_task.items():
        max_level_task = max(tasks, key=itemgetter('m_iCTRReqLvMin'))
        time_frac = growth_frac[max_level_task['m_iCTRReqLvMin']]
        src_loop_time = TIME_TO_LV36 * time_frac 
        time_till_item = src_loop_time / len(tasks)

        secs_per_mission_rewards.merge(key, {
            **item_metadata(key),
            'src_id': max_level_task['m_iHMissionID'],
            'src_name': f"{MISSION_TYPES[max_level_task['m_iHMissionType']]}: {max_level_task['m_strName']}{('' if len(tasks) == 1 else ' and {} more'.format(len(tasks) - 1))}",
            'srcs_in_loop': len(tasks),
            'src_loop_time': src_loop_time,
            'time_till_item': time_till_item,
            'approx_taros': seconds_to_taros(time_till_item),
        })

    return secs_per_mission_rewards


def get_crate_count(t_per_crates):
    return sum([itype == CRATE_TYPE_ID for _, itype in t_per_crates])


def merge_taros_per_crate_contents(t_per_crates: MergeDict):
    t_per_crates_new = MergeDict(fitness_key=t_per_crates.fitness_key)

    for (iid, itype), info in t_per_crates.items():
        t_per_crates_new.merge((iid, itype), info)

        if itype != CRATE_TYPE_ID:
            continue

        for irid, best_chance in drops_per_crate(iid).items():
            key = itemref_to_id_type[irid]

            if best_chance > 1e-7:
                info_item = info.copy()
                time_till_item = info_item['time_till_item'] / best_chance
                
                info_item.update({
                    **item_metadata(key),
                    'time_till_item': time_till_item,
                    'approx_taros': seconds_to_taros(time_till_item),
                })

                t_per_crates_new.merge(key, info_item)

    return t_per_crates_new


def merge_all_taros_per_crate_contents(t_per_crates):
    t_per_crates_new = t_per_crates

    while True:
        prev_crates_opened = get_crate_count(t_per_crates_new)
        t_per_crates_new = merge_taros_per_crate_contents(t_per_crates_new)
        crates_opened = get_crate_count(t_per_crates_new)

        if crates_opened == prev_crates_opened:
            break

    return t_per_crates_new


def crate_content_effect(sources: MergeDict, source_names=frozenset()):
    sources_new = MergeDict(fitness_key='approx_taros')

    for (iid, itype), info in sources.items():
        sources_new.merge((iid, itype), info)

        if itype != CRATE_TYPE_ID or iid not in drop_maps['Crates'] or all([source_name not in info['src_name'] 
                                                                            for source_name in source_names]):
            continue

        crate_content_price = np.sum([best_chance * sources[itemref_to_id_type[irid]]['approx_taros']
                                      for irid, best_chance in drops_per_crate(iid).items()]).astype(int)
        
        info_crate = info.copy()
        info_crate['approx_taros'] = crate_content_price
        sources_new.merge((iid, itype), info_crate)

    return sources_new


def fast_first_item_effect(sources: MergeDict, source_names=frozenset(), factor=1.0):
    sources_new = MergeDict(fitness_key='approx_taros')

    for (iid, itype), info in sources.items():
        sources_new.merge((iid, itype), info)

        if all([source_name not in info['src_name'] for source_name in source_names]):
            continue
        
        time_till_item = info['time_till_item'] * factor
        info_item = info.copy()
        info_item['approx_taros'] = seconds_to_taros(time_till_item)
        sources_new.merge((iid, itype), info_item)

    return sources_new


def rarity_relevance_effect(sources: MergeDict, source_names=frozenset(), factors=None):
    factors = factors or [1.0, 1.0, 1.0, 1.0, 1.0]
    sources_new = MergeDict(fitness_key='approx_taros')

    for (iid, itype), info in sources.items():
        sources_new.merge((iid, itype), info)

        if all([source_name not in info['src_name'] for source_name in source_names]):
            continue
        
        time_till_item = info['time_till_item'] * factors[RARITY.index(info['rarity'])]
        info_item = info.copy()
        info_item['approx_taros'] = seconds_to_taros(time_till_item)
        sources_new.merge((iid, itype), info_item)

    return sources_new

# Path Research

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
plt.imshow(MINIMAP)

for id, mob_info in mobmap.items():
    color = mob_info['color']
    x, y = extract_coordinates(mob_info['mobs'])[:2]

    plt.scatter(pixel(x), rpixel(y), s=1, color=color)
    plt.annotate(str(id), (pixel(x.mean()), rpixel(y.mean())), color=color, fontsize='x-small')

plt.show()

In [ ]:
id = 433
x, y, z, ins = extract_coordinates(mobmap[id]['mobs'])
dist, zdist = get_masked_dists(x, y, z, ins)
ctups = get_mst(dist, zdist)

cols = 3
rows = int(np.ceil(len(ctups) / cols))
colors = {ci: (np.random.random(), np.random.random(), np.random.random()) for ci in range(len(ctups))}

fig = plt.figure(figsize=(10 * cols, 10 * rows))

for pi in range(len(ctups)):
    ax = fig.add_subplot(rows, cols, pi + 1, projection='3d')
    
    for ci, (conn_idx, conn_tups) in enumerate(ctups):
        conn_arr = np.array(list(conn_idx), dtype=np.int64)
        xconn = x[conn_arr]
        yconn = y[conn_arr]
        zconn = z[conn_arr]

        if pi == ci:
            xcurr = xconn
            ycurr = yconn
            zcurr = zconn

        ax.scatter(pixel(xconn), rpixel(yconn), pixel(zconn), color=colors[ci])
        ax.text(pixel(xconn.mean()), rpixel(yconn.mean()), pixel(zconn.mean()), f'{id}-{ci}', color=colors[ci])

        for i, j in conn_tups:
            ax.plot3D([pixel(x[i]), pixel(x[j])], [rpixel(y[i]), rpixel(y[j])], [pixel(z[i]), pixel(z[j])], color=colors[ci])

    ax.set_xlim3d(pixel(xcurr.min()), pixel(xcurr.max()))
    ax.set_ylim3d(rpixel(ycurr.min()), rpixel(ycurr.max()))
        
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(MINIMAP)

for id, mob_info in mobmap.items():
    focmob = mob_info['mobs']
    color = mob_info['color']
    
    x, y, z, ins = extract_coordinates(focmob)
    dist, zdist = get_masked_dists(x, y, z, ins)
    ctups = get_mst(dist, zdist)

    for ci, (conn_idx, conn_tups) in enumerate(ctups):
        for i, j in conn_tups:
            ax.plot([pixel(x[i]), pixel(x[j])], [rpixel(y[i]), rpixel(y[j])], color=color)

        conn_arr = np.array(list(conn_idx), dtype=np.int64)
        ax.annotate(f'{id}-{ci}', (pixel(x[conn_arr].mean()), rpixel(y[conn_arr].mean())), color=color, fontsize='x-small')

plt.show()


In [ ]:
id = 2458
x, y, z, ins = extract_coordinates(mobmap[id]['mobs'])
dist, zdist = get_masked_dists(x, y, z, ins)
tsps = get_tsp_ordering(dist, zdist)

cols = 3
rows = int(np.ceil(len(tsps) / cols))
colors = {ci: (np.random.random(), np.random.random(), np.random.random()) for ci in range(len(tsps))}

fig = plt.figure(figsize=(10 * cols, 10 * rows))

for pi in range(len(tsps)):
    ax = fig.add_subplot(rows, cols, pi + 1, projection='3d')

    for ci, tsp in enumerate(tsps):
        xs = [x[tsp[0]]]
        ys = [y[tsp[0]]]
        zs = [z[tsp[0]]]

        for node in tsp[1:]:
            xs.append(x[node])
            ys.append(y[node])
            zs.append(z[node])
            ax.plot3D([pixel(xs[-2]), pixel(xs[-1])], [rpixel(ys[-2]), rpixel(ys[-1])], [pixel(zs[-2]), pixel(zs[-1])], color=colors[ci])

        if len(xs) > 1:
            xs.pop()
            ys.pop()
            zs.pop()
        
        xconn = np.array(xs)
        yconn = np.array(ys)
        zconn = np.array(zs)

        if pi == ci:
            xcurr = xconn
            ycurr = yconn
            zcurr = zconn

        ax.scatter(pixel(xconn), rpixel(yconn), pixel(zconn), color=colors[ci])
        ax.text(pixel(xconn.mean()), rpixel(yconn.mean()), pixel(zconn.mean()), f'{id}-{ci}', color=colors[ci])
    
    ax.set_xlim3d(pixel(xcurr.min()), pixel(xcurr.max()))
    ax.set_ylim3d(rpixel(ycurr.min()), rpixel(ycurr.max()))
    #ax.set_zlim3d(xpixel(zcurr.min()), xpixel(zcurr.max()))

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(MINIMAP)

for id, mob_info in mobmap.items():
    focmob = mob_info['mobs']
    color = mob_info['color']
    
    x, y, z, ins = extract_coordinates(focmob)
    dist, zdist = get_masked_dists(x, y, z, ins)
    tsps = get_tsp_ordering(dist, zdist)

    for ci, tsp in enumerate(tsps):
        xs = [x[tsp[0]]]
        ys = [y[tsp[0]]]

        for node in tsp[1:]:
            xs.append(x[node])
            ys.append(y[node])
            ax.plot([pixel(xs[-2]), pixel(xs[-1])], [rpixel(ys[-2]), rpixel(ys[-1])], color=color)

        if len(xs) > 1:
            xs.pop()
            ys.pop()
        
        ax.annotate(f'{id}-{ci}', (pixel(np.mean(xs)), rpixel(np.mean(ys))), color=color, fontsize='x-small')

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(MINIMAP)

for id, mob_info in mobmap.items():
    focmob = mob_info['mobs']
    color = mob_info['color']

    x, y, z, ins = extract_coordinates(focmob)
    dist, zdist = get_masked_dists(x, y, z, ins)
    tsps = get_tsp_ordering(dist, zdist)
    tsp, mob_loop_time = best_loop(tsps, dist, focmob)

    xs = [x[tsp[0]]]
    ys = [y[tsp[0]]]

    for node in tsp[1:]:
        xs.append(x[node])
        ys.append(y[node])
        ax.plot([pixel(xs[-2]), pixel(xs[-1])], [rpixel(ys[-2]), rpixel(ys[-1])], color=color)

    if len(xs) > 1:
        xs.pop()
        ys.pop()
    
    ax.annotate(f'{id}-{int(mob_loop_time / 60)}m{int(mob_loop_time % 60)}s', (pixel(np.mean(xs)), rpixel(np.mean(ys))), color=color, fontsize='x-small')

plt.show()

# Source Discovery

In [25]:
def make_dataframe(sources, index=None):
    df = pd.DataFrame.from_records(list(sources.values()), index=index)

    for time_field in ['time_till_misc', 'time_till_item', 'src_loop_time']:
        if time_field in df.columns:
            df[time_field] = df[time_field].transform(pd.Timedelta, unit='sec')

    return df


best_misc = fastest_miscs()
make_dataframe(best_misc).to_csv('best_misc.csv', sep=';')
taros_per_second = int(round(1 / best_misc['Taro']['time_till_misc']))

best_sources = MergeDict(fitness_key='approx_taros')

funcs = [
    # Code Item Data
    taros_per_code_items,
    # Vendor Data
    taros_per_vendor_items,
    # Egg Data
    seconds_per_egg_items,
    # Racing Data
    seconds_per_racing_crates,
    # Mission Data
    seconds_per_mission_rewards,
    # Drop Data
    seconds_per_mob_drops,
    # Event Data
    seconds_per_event_drops,
]

for func in funcs:
    sources = merge_all_taros_per_crate_contents(func())
    best_sources.merge_from(sources)
    make_dataframe(sources, index=['id', 'type']).sort_values(by=['tradable', 'approx_taros'], ascending=False).to_csv(f'{func.__name__}.csv', sep=';')

make_dataframe(best_sources, index=['id', 'type']).sort_values(by=['tradable', 'approx_taros'], ascending=False).to_csv('best_sources.csv', sep=';')

effect_best_sources = best_sources
effect_best_sources = fast_first_item_effect(effect_best_sources, source_names=['Mission:', 'Code:', 'Egg:'], factor=0.2)
effect_best_sources = rarity_relevance_effect(effect_best_sources, source_names=['Mob:'], factors=[1.0, 0.05, 0.1, 0.2, 1.0])
effect_best_sources = crate_content_effect(effect_best_sources, source_names=['Mission:', 'Mob:'])

make_dataframe(effect_best_sources, index=['id', 'type']).sort_values(by=['tradable', 'approx_taros'], ascending=False).to_csv('effect_best_sources.csv', sep=';')


Fastest CRATE rate: 134 Motorilla 2.3333333333333335 seconds per CRATE
